In [ ]:
# Initial development for DicomFile

# Import classes and methods
from pybrors.dicom import DicomFile

# Create file instance
tmp = DicomFile(file_path="../data/dicom/dicom_img_file.dcm")
print(tmp)
print(tmp.dataset["PatientName"].value)

# Anonymize DICOM file
if tmp.anonymize():
    print("DICOM has been anonymized.")
else:
    print("DICOM has not been anonymized.")

# Display anonymized file information
print(tmp)
print(tmp.dataset["PatientName"])

In [ ]:
# Initial development for DicomDir

# Import classes and methods
from pybrors.dicom import DicomDir

# Create file instance
tmp = DicomDir(dir_path="../data/dicom")
print(tmp)
print(tmp.dicom_df)

# Anonymize DICOM file
if tmp.anonymize():
    print("DICOM directory has been anonymized.")
else:
    print("DICOM directory has not been anonymized.")